In [61]:
#import packages and data
import pandas as pd
import random
random.seed(30)
df3_clean = pd.read_csv("second_data1709.csv", header = 0)
df3_additional = pd.read_csv("df3addition_data1709.csv", header = 0)
df3_additional = df3_additional.loc[(df3_additional["num_in_trans"]!= 0)&(df3_additional["num_out_trans"]!= 0)]
EL_final = pd.read_csv("EL_only_origin.csv", header = 0)
EL_additional = pd.read_csv("EL_only_origin_add0709.csv", header = 0)
EL_additional = EL_additional.dropna()
EL_clean = EL_final.dropna()
corrected_numOUT = pd.read_csv("corrected_numoutsins.csv", header = 0)


In [62]:
#solve a bug (OUT was wrongly defined before and it was easier to just create the right column again instead of doing everything from scratch again)
df3_clean.num_in_trans = corrected_numOUT.num_in_trans
df3_clean.num_out_trans = corrected_numOUT.num_out_trans

## cleaning

In [63]:
df3_clean = df3_clean.loc[(df3_clean["num_in_trans"]!= 0)&(df3_clean["num_out_trans"]!= 0)]
df3_clean = df3_clean.loc[df3_clean["wallet_address"]!= "an address"] #delete all observations without meaningful transactions as they just lead to many NAs (in the real application one can create anot

In [64]:
df3_clean
df3_one = df3_clean.drop(columns = ["address_balance",  "Unnamed: 0", "address", "full_name", "address_length", "hash", "address_beginning"])

In [65]:
df3_two = df3_additional.drop(columns = [ "Unnamed: 0", "address", "full_name", "address_length", "address_beginning", 'num_out_trans_1neighb', 'avg_tx_out_neighb',
       'avg_tx_in_neighb'])

In [68]:
df3_one.columns == df3_two.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True])

In [69]:
df3_final = df3_one.append(df3_two, ignore_index=True)
df3_final.columns

Index(['Unnamed: 0.1', 'sow_estimated_net_worth_value', 'wallet_address',
       'num_in_trans', 'num_out_trans', 'avg_trans_out', 'avg_trans_in',
       'sum_trans_in', 'sum_trans_out', 'max_trans_out', 'max_trans_in',
       'std_trans_in', 'std_trans_out', 'min_trans_out', 'min_trans_in',
       'unique_tpartn_out', 'unique_tpartn_in', 'avg_transcost_out',
       'avg_transcost_in', 'max_transcost_out', 'max_transcost_in',
       'std_transcost_out', 'std_transcost_in', 'timedelta_trans_out',
       'timedelta_trans_in', 'duration_out', 'num_in_trans_1neighb',
       'avg_oneNightStands_out', 'avg_oneNightStands_in', 'max_tx_out_neighb',
       'max_tx_in_neighb', 'sum_tx_in_neighb', 'sum_tx_out_neighb',
       'std_tx_out_neighb', 'std_tx_in_neighb', 'unique_tx_out_neighb',
       'unique_tx_in_neighb', 'avg_txcost_out_neighb', 'avg_txcost_in_neighb',
       'max_txcost_in_neighb', 'max_txcost_out_neighb', 'num_in_transERC',
       'num_out_transERC', 'ERCunique_tpartn_out', 'ERCun

In [70]:
df_final = df3_final.drop(columns =  ["avg_txcost_in_neighb","Unnamed: 0.1", "std_tx_in_neighb", "std_transcost_in", "max_transcost_in", "std_tx_in_neighb", "unique_tx_in_neighb", "timedelta_trans_out", "timedelta_trans_in", "max_txcost_in_neighb", "sum_trans_out"])
df_final.columns

Index(['sow_estimated_net_worth_value', 'wallet_address', 'num_in_trans',
       'num_out_trans', 'avg_trans_out', 'avg_trans_in', 'sum_trans_in',
       'max_trans_out', 'max_trans_in', 'std_trans_in', 'std_trans_out',
       'min_trans_out', 'min_trans_in', 'unique_tpartn_out',
       'unique_tpartn_in', 'avg_transcost_out', 'avg_transcost_in',
       'max_transcost_out', 'std_transcost_out', 'duration_out',
       'num_in_trans_1neighb', 'avg_oneNightStands_out',
       'avg_oneNightStands_in', 'max_tx_out_neighb', 'max_tx_in_neighb',
       'sum_tx_in_neighb', 'sum_tx_out_neighb', 'std_tx_out_neighb',
       'unique_tx_out_neighb', 'avg_txcost_out_neighb',
       'max_txcost_out_neighb', 'num_in_transERC', 'num_out_transERC',
       'ERCunique_tpartn_out', 'ERCunique_tpartn_in', 'ERCavg_transcost_out',
       'ERCavg_transcost_in', 'ERCmax_transcost_out', 'ERC_nb_tkn_tot',
       'high_pressure_sells', 'high_pressure_buys'],
      dtype='object')

In [74]:
df_final = df_final.fillna(0) #replace the NAs with zeros as the only emerge because there were no such transactions that the API could have mentioned

In [ ]:
#df_final this is the final data set without node embeddings

## create embeddings

In [44]:
EL_clean = EL_clean.drop(columns="Unnamed: 0")

In [46]:
EL_clean = EL_clean.append(EL_additional, ignore_index= True) #append the additional edge list of the 300 extra sampled addresses

In [2]:
#pip install node2vec --upgrade

In [216]:
#pip install --upgrade node2vec

In [48]:
import networkx as nx
graph = nx.from_pandas_edgelist(EL_clean, 'tx_from', 'tx_to', create_using=nx.DiGraph())#do it without weights as some are 0 and that´s not possible. ecause weights (after normalization) are treated as walk probabilities, so essentially using the weights you can control how strongly two nodes are connected.

In [49]:
print(nx.info(graph))

Name: 
Type: DiGraph
Number of nodes: 31295
Number of edges: 75813
Average in degree:   2.4225
Average out degree:   2.4225


In [26]:
#pip install numpy


Note: you may need to restart the kernel to use updated packages.


In [50]:
from node2vec import Node2Vec

In [51]:

# Precompute probabilities and generate walks - **ON WINDOWS ONLY WORKS WITH workers=1** ##hsnr jetzt auf 12 dimensions stat 32. ggf besser wegen overfitting
node2vec_10 = Node2Vec(graph, dimensions=10, walk_length=10, num_walks=200, p = 1.1, q = 0.9, workers=4, temp_folder = "/Users/Kenneth/Desktop/crypto/BA_data_science")  # Use temp_folder for big graphs #q>p damit mehr die Strukturellen Ähnlichkeiten untersucht werden als die Communities

# Embed nodes
model_10 = node2vec_10.fit(window=10, min_count=1, batch_words=5)  # Any keywords acceptable by gensim.Word2Vec can be passed, `dimensions` and `workers` are automatically passed (from the Node2Vec constructor)

# Look for most similar nodes
#model.wv.most_similar('2')  # Output node names are always strings

# Save embeddings for later use
#model.wv.save_word2vec_format("embedding.txt")

# Save model for later use
#model.save("model.txt")



Generating walks (CPU: 1):   0%|          | 0/50 [00:00<?, ?it/s]



Generating walks (CPU: 1):   4%|▍         | 2/50 [00:10<04:07,  5.16s/it]




Generating walks (CPU: 1):   6%|▌         | 3/50 [00:20<05:15,  6.71s/it]


Generating walks (CPU: 1):   8%|▊         | 4/50 [00:30<05:57,  7.77s/it]


Generating walks (CPU: 1):  10%|█         | 5/50 [00:41<06:24,  8.55s/it]


Generating walks (CPU: 1):  12%|█▏        | 6/50 [00:51<06:33,  8.95s/it]


Generating walks (CPU: 1):  14%|█▍        | 7/50 [01:01<06:38,  9.26s/it]


Generating walks (CPU: 1):  16%|█▌        | 8/50 [01:11<06:38,  9.48s/it]


Generating walks (CPU: 1):  18%|█▊        | 9/50 [01:21<06:41,  9.80s/it]


Generating walks (CPU: 1):  20%|██        | 10/50 [01:31<06:33,  9.84s/it]


Generating walks (CPU: 1):  22%|██▏       | 11/50 [01:41<06:29,  9.98s/it]

Generating walks (CPU: 1):  26%|██▌       | 13/50 [02:01<06:08,  9.95s/it]





Generating walks (CPU: 1):  28%|██▊       | 14/50 [02:12<06:03, 10.10s/it]


Generating 

In [54]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
#import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn import linear_model
from sklearn.model_selection import cross_validate

In [55]:
nodes = df3_final.wallet_address

hi = []
for node in nodes: 
    x= pd.DataFrame(model_10.wv[str(node)])
    y = x.transpose()
    y["wallet_address"] = node
    hi.append(y)
    final_6 = pd.concat(hi)
final_6

,0,1,2,3,4,5,6,7,8,9,wallet_address
0,0.635960,-1.053419,0.879687,-1.638524,1.752925,1.240897,1.205985,0.631892,-0.443489,0.389424,0x3296b13b1ee66d953d4449bd81581a4612ecc27f
0,2.255808,-1.231589,2.259259,-0.876529,2.564296,0.110971,0.912556,0.505581,-0.595875,-1.290875,0xaa9d5a6cdb9344fd22cda14e5de5c1de9c41cc91
0,1.102425,0.197649,1.677485,-1.133659,1.892117,-0.520115,0.426440,0.988850,-1.431002,-1.172790,0x9329c22f63898e18e8a67c582fe8092203b90c64
0,0.217237,-0.388497,1.979959,-0.662067,1.505269,0.497690,1.033385,1.863815,0.043531,-0.544956,0xbefb4fecd83185d358b891f12eda1c20fdba9c20
0,0.559830,-2.128753,1.077548,-0.303532,1.590936,-1.225339,1.087649,-0.203427,-1.147875,-0.470743,0x88c4181152f4133a7a63cabee0c601f75ddeb867
...,...,...,...,...,...,...,...,...,...,...,...
0,0.382709,-0.810958,0.565752,-0.179420,0.773218,0.392337,-0.307832,0.690635,-1.284273,-0.649465,0xecaf77df98dc538ea057ad11e8651e01e97a9628
0,1.950280,-0.655413,-0.082548,-1.012925,1.299750,0.697862,0.902658,0.014000,-0.817201,-0.427554,0xb2f1a288e6bf497e3b41adf23a4a89896069d1f6
0,1.086755,-0.879203,0.993179,-0.917366,1.118462,0.749132,0.613680,1.413900,-0.276596,0.431619,0x42afe93a50c9fba4bf55de264da7bfacdbd2ec70
0,0.768667,-2.143785,1.916566,2.996222,-0.799109,0.354020,0.918835,-0.212702,-4.358207,-0.281058,0x1dd3fdd3b637b88c4a2387199769ca1adb23847c


In [76]:
final_6.to_csv("2709_08node_embeddings.csv")
#final.to_csv("0709node_embeddings.csv")

In [78]:
final_6 = pd.read_csv("2709_10node_embeddings.csv", header = 0)

In [79]:
result = pd.merge(df_final, final_6, on = ["wallet_address"])

In [80]:
modelling_data = result.drop(columns = ["wallet_address"])

In [81]:
modelling_data = modelling_data.drop(columns="Unnamed: 0")

In [82]:
modelling_data = modelling_data.dropna()

In [193]:
pd.DataFrame(modelling_data).isnull().sum()

sow_estimated_net_worth_value    0
num_in_trans                     0
num_out_trans                    0
avg_trans_out                    0
avg_trans_in                     0
sum_trans_in                     0
max_trans_out                    0
max_trans_in                     0
std_trans_in                     0
std_trans_out                    0
min_trans_out                    0
min_trans_in                     0
unique_tpartn_out                0
unique_tpartn_in                 0
avg_transcost_out                0
avg_transcost_in                 0
max_transcost_out                0
std_transcost_out                0
duration_out                     0
num_in_trans_1neighb             0
avg_oneNightStands_out           0
avg_oneNightStands_in            0
max_tx_out_neighb                0
max_tx_in_neighb                 0
sum_tx_in_neighb                 0
sum_tx_out_neighb                0
std_tx_out_neighb                0
unique_tx_out_neighb             0
avg_txcost_out_neigh

## Create X & Y Train/Test Data Set

In [84]:
train_df, test_df = train_test_split(modelling_data, test_size = 0.25, random_state = 0)

In [86]:
train_df.sow_estimated_net_worth_value.sum()/len(train_df.sow_estimated_net_worth_value)

0.267260579064588

In [87]:
len(train_df.sow_estimated_net_worth_value)

449

In [88]:

X_train = train_df.loc[:, modelling_data.columns != "sow_estimated_net_worth_value"]
y_train = train_df.sow_estimated_net_worth_value
X_test = test_df.loc[:, modelling_data.columns != "sow_estimated_net_worth_value"]
y_test = test_df.sow_estimated_net_worth_value

In [176]:
df_final

,sow_estimated_net_worth_value,wallet_address,num_in_trans,num_out_trans,avg_trans_out,avg_trans_in,sum_trans_in,max_trans_out,max_trans_in,std_trans_in,...,num_in_transERC,num_out_transERC,ERCunique_tpartn_out,ERCunique_tpartn_in,ERCavg_transcost_out,ERCavg_transcost_in,ERCmax_transcost_out,ERC_nb_tkn_tot,high_pressure_sells,high_pressure_buys
0,0.0,0x3296b13b1ee66d953d4449bd81581a4612ecc27f,76,1253,612.922650,3272.342760,248698.049792,36886.410000,38954.841000,6898.080060,...,609,591,170,185,15.146850,22.514325,259.066805,241,9,10
1,0.0,0xaa9d5a6cdb9344fd22cda14e5de5c1de9c41cc91,3,9,684.060067,2070.885533,6212.656600,5869.750000,6189.669450,3566.990022,...,2,1,1,1,3.445493,1.165070,3.445493,2,0,0
2,0.0,0x9329c22f63898e18e8a67c582fe8092203b90c64,5,36,138.577080,318.081610,1590.408052,1171.300000,1172.300000,506.507387,...,11,1,1,9,4.609975,4.558068,4.609975,9,5,0
3,0.0,0xbefb4fecd83185d358b891f12eda1c20fdba9c20,23,267,42.425081,187.562190,4313.930359,4563.124500,1047.439086,270.530960,...,148,133,43,59,9.468765,7.778343,98.525757,75,7,0
4,0.0,0x88c4181152f4133a7a63cabee0c601f75ddeb867,24,50,59.081390,94.514031,2268.336747,458.597876,1062.141776,214.331326,...,24,12,9,17,3.586294,6.145770,16.322677,16,7,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
568,0.0,0xecaf77df98dc538ea057ad11e8651e01e97a9628,3,1,1221.947089,728.571352,2185.714056,1221.947089,1212.983228,419.514923,...,0,0,0,0,NaN,NaN,NaN,0,0,0
569,0.0,0xb2f1a288e6bf497e3b41adf23a4a89896069d1f6,58,505,601.602494,1613.747810,93597.373008,48565.950000,15068.412760,2664.224359,...,204,175,93,99,27.046761,41.962073,203.094550,110,8,8
570,0.0,0x42afe93a50c9fba4bf55de264da7bfacdbd2ec70,102,793,136.241588,384.681676,39237.530941,5927.872867,5409.802813,819.502120,...,312,287,103,122,14.608937,17.551375,89.073868,156,7,7
571,0.0,0x1dd3fdd3b637b88c4a2387199769ca1adb23847c,33,196,1.346290,25.520927,842.190590,100.947000,145.510786,39.753041,...,197,54,45,123,7.010335,14.258410,105.270845,111,9,8


In [98]:
X_train.to_csv("X_trainNonOver_8_3.csv")
y_train.to_csv("y_trainNonOver_8_3.csv")
X_test.to_csv("X_testNonOver_8_3.csv")
y_test.to_csv("y_testNonOver_8_3.csv")